In [1]:
import sqlite3
import time
import statistics

In [2]:
db_path = 'Dirigo.sqlite'

query = '''
WITH IdentifiedTruck AS (
  SELECT Object_id AS TruckID
  FROM OCEL_Truck
  WHERE LPT = '926VFT'
),
PickupTimestamp AS (
  SELECT MIN(Timestamp) AS FirstPickupTime
  FROM OCEL_Truck
  WHERE Object_id = (SELECT TruckID FROM IdentifiedTruck)
    AND "Cargo ID" = 'Cr1'
),
TruckForPickupPlan AS (
  SELECT "Pickup Plan ID"
  FROM OCEL_Truck
  WHERE Object_id = (SELECT TruckID FROM IdentifiedTruck)
  AND Timestamp = (SELECT FirstPickupTime FROM PickupTimestamp)
  AND "Pickup Plan ID" IS NOT NULL
  ),
PickupPlanEventWindow AS (
  SELECT MIN(Timestamp) AS StartTime, MAX(Timestamp) AS EndTime
  FROM OCEL_O2O 
  WHERE Source_object_id = (SELECT TruckID FROM IdentifiedTruck)
  AND Target_object_id = (SELECT "Pickup Plan ID" FROM TruckForPickupPlan)
),
TruckWeightEventsInPickupWindow AS (
  SELECT *
  FROM OCEL_Truck
  WHERE object_id = (SELECT TruckID FROM IdentifiedTruck)
    AND Ocel_changed_field = 'Truck Weight'
    AND Timestamp BETWEEN 
        (SELECT StartTime FROM PickupPlanEventWindow)
        AND 
        (SELECT EndTime FROM PickupPlanEventWindow)
),
EmptyWeighingTime AS (
SELECT MIN(Timestamp) AS Empty_weighingtime
FROM TruckWeightEventsInPickupWindow
)
SELECT "Truck Weight"
FROM OCEL_Truck
WHERE Timestamp = (SELECT Empty_weighingtime FROM EmptyWeighingTime);
'''
repeat = 1000

conn = sqlite3.connect(db_path)
cursor = conn.cursor()


durations = []

for i in range(repeat):
    start_time = time.time()
    cursor.execute(query)
    cursor.fetchall()  
    end_time = time.time()
    
    duration = (end_time - start_time) * 1000  
    durations.append(duration)
    # print(f"Run {i+1}: {duration:.4f} ms")


avg = statistics.mean(durations)
std_dev = statistics.stdev(durations)

print("\n=== Execution Statistics (Dirigo) ===")
print(f"Average execution time: {avg:.4f} ms")
print(f"Standard deviation: {std_dev:.4f} ms")


cursor.close()
conn.close()


=== Execution Statistics (Dirigo) ===
Average execution time: 0.2161 ms
Standard deviation: 0.0495 ms


In [3]:
db_path = 'ACEL.sqlite'

query = """
WITH IdentifiedTruck AS (
  SELECT ObjectId AS TruckID
  FROM ACEL_Objects
  WHERE LPT = '926VFT'
),
TruckEvents AS (
SELECT e.*
FROM ACEL_Events e
WHERE EXISTS (
  SELECT 1
  FROM json_each(REPLACE(e.Objects, '''', '"'))
  WHERE json_each.value = (SELECT TruckID FROM IdentifiedTruck))
 ),
 TruckPickupRelation As (
SELECT Relations
FROM TruckEvents
WHERE EXISTS (
   SELECT 1
   FROM json_each(REPLACE(ObjectChanges, '''', '"')) AS jc
  WHERE json_extract(jc.value, '$.Attribute') = 'Cargo ID'
    AND json_extract(jc.value, '$.NewValue') = 'Cr1')
),
FilteredEvents As (
SELECT * 
FROM ACEL_Events
WHERE Relations = (SELECT Relations FROM TruckPickupRelation)
),
Timerange AS (
SELECT
  MIN(Timestamp) AS StartTime,
  MAX(Timestamp) AS EndTime
  FROM FilteredEvents
 ),
 EmptyWeighing AS (
 SELECT *
 FROM TruckEvents
 WHERE Activity = 'Weigh the Empty Truck'
 AND Timestamp Between 
 (SELECT StartTime FROM Timerange)
  AND 
 (SELECT EndTime FROM Timerange)
 )
 SELECT json_extract(jc.value, '$.NewValue') AS NewWeight
FROM EmptyWeighing, json_each(REPLACE(ObjectChanges, '''', '"')) AS jc
WHERE json_extract(jc.value, '$.Attribute') = 'Truck Weight';
"""

repeat = 1000

conn = sqlite3.connect(db_path)
cursor = conn.cursor()


durations = []

for i in range(repeat):
    start_time = time.time()
    cursor.execute(query)
    cursor.fetchall()  
    end_time = time.time()
    
    duration = (end_time - start_time) * 1000  
    durations.append(duration)
    # print(f"Run {i+1}: {duration:.4f} ms")


avg = statistics.mean(durations)
std_dev = statistics.stdev(durations)

print("\n=== Execution Statistics (ACEL) ===")
print(f"Average execution time: {avg:.4f} ms")
print(f"Standard deviation: {std_dev:.4f} ms")


cursor.close()
conn.close()


=== Execution Statistics (ACEL) ===
Average execution time: 0.2637 ms
Standard deviation: 0.0394 ms


In [4]:
db_path = 'DOCEL.sqlite'

query = """
 WITH IdentifiedTruck AS (
  SELECT ObjectID AS TruckID
  FROM DOCEL_Truck_Static
  WHERE LPT = '926VFT'
),
TruckEvents AS (
  SELECT * 
  FROM DOCEL_Events
 WHERE EXISTS (
  SELECT 1
  FROM json_each(REPLACE(Truck, '''', '"'))
  WHERE json_each.value = (SELECT TruckID FROM IdentifiedTruck))
  ORDER BY Timestamp
 ),
 TruckAssignEvent AS (
 SELECT eventID
 FROM DOCEL_TruckCargoID
 WHERE objectID = (SELECT TruckID FROM IdentifiedTruck)
 AND "Cargo ID" = 'Cr1'
 ),
 AssignEventTime AS (
  SELECT Timestamp
  FROM DOCEL_Events
  WHERE eventID = (SELECT eventID FROM TruckAssignEvent)
),
 NextAssignment AS (
SELECT e.eventID, e.Timestamp
  FROM DOCEL_Events e
  WHERE e.Activity = 'Assign Truck'
    AND EXISTS (
      SELECT 1
      FROM json_each(REPLACE(e.Truck, '''', '"'))
      WHERE json_each.value = (SELECT TruckID FROM IdentifiedTruck)
    )
    AND e.Timestamp > (SELECT Timestamp FROM AssignEventTime)
  ORDER BY e.Timestamp ASC
  LIMIT 1
  ),
  EmptyWeighingEvent AS (
 SELECT eventID
 FROM TruckEvents
 WHERE Activity = 'Weigh the Empty Truck'
 AND Timestamp BETWEEN
  (SELECT Timestamp FROM AssignEventTime)
  AND 
 (SELECT Timestamp FROM NextAssignment)
 )
 SELECT "Truck Weight" 
 FROM DOCEL_TruckWeight tw
 WHERE tw.eventID = (SELECT eventID FROM EmptyWeighingEvent);
"""
repeat = 1000

conn = sqlite3.connect(db_path)
cursor = conn.cursor()


durations = []

for i in range(repeat):
    start_time = time.time()
    cursor.execute(query)
    cursor.fetchall()  
    end_time = time.time()
    
    duration = (end_time - start_time) * 1000  
    durations.append(duration)
    # print(f"Run {i+1}: {duration:.4f} ms")


avg = statistics.mean(durations)
std_dev = statistics.stdev(durations)

print("\n=== Execution Statistics (DOCEL) ===")
print(f"Average execution time: {avg:.4f} ms")
print(f"Standard deviation: {std_dev:.4f} ms")


cursor.close()
conn.close()


=== Execution Statistics (DOCEL) ===
Average execution time: 0.2999 ms
Standard deviation: 0.0653 ms
